In [1]:
import os
import sys
from pathlib import Path
#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)


In [2]:

import json
import cv2
import tensorflow as tf
import numpy as np
import PIL.Image as Image

import Tools.datasetTools as DatasetTools
import Tools.utilities as common_utils

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


2023-10-05 15:54:42.642681: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-05 15:54:42.667788: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 15:54:42.667823: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 15:54:42.667840: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 15:54:42.672597: I tensorflow/core/platform/cpu_feature_g

2.14.0
len(devices):  0
available GPUs: []


2023-10-05 15:54:45.051130: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-10-05 15:54:45.051162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: nalain-dell
2023-10-05 15:54:45.051168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: nalain-dell
2023-10-05 15:54:45.051253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.113.1
2023-10-05 15:54:45.051272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-10-05 15:54:45.051277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.113.1


**Dataloaders**

In [3]:
import data_setup


# try to import the dataset
dataset="SkyData"
BATCH_SIZE = 1
SHUFFLE_BUFFER_SIZE = 100

print("**dataset: ", dataset)
train_dataloader,test_dataloader = data_setup.create_dataloaders(dataset=dataset, 
                                                                BATCH_SIZE=BATCH_SIZE,
                                                                SHUFFLE_BUFFER_SIZE=100
                                                                )
#

**dataset:  SkyData
[INFO] loading train dataset
[INFO] train _dataset:  27700
[INFO] loading val dataset
[INFO] val _dataset:  7990


**Dataset integrity**
- display samples count
- samples(images) don't contain NaNs
- inverse of homography matrix exits


In [4]:
dataloaders = {
    "train":train_dataloader,
    "test":test_dataloader
}

dataset_info={}
dataset_info["dataset"]= dataset


In [5]:
for split, dataloader in dataloaders.items():
    dataset_info[split]={}
    dataset_info[split]["size"] = len(dataloader)
    dataset_info[split]["NaNs"] = []
    dataset_info[split]["transformed_have_NaNs"] = []
    dataset_info[split]["nonInvertibles"] = []
    dataset_info[split]["input_image_size"] = str(list(dataloader.take(1))[0][0].shape)
    dataset_info[split]["template_image_size"] = str(list(dataloader.take(1))[0][1].shape)
    
    for batch in dataloader.take(1):
        input_images, template_images, labels ,_instances = batch
        
        # check for nans
        if np.isnan(input_images).any() or np.isnan(template_images).any():
            dataset_info[split]["NaNs"].append(str(_instances.numpy()[0]))
            
        homography_matrices = DatasetTools.get_ground_truth_homographies(labels) 
        inverse_homography_matrices, _is_invertibles = DatasetTools.is_invertible(homography_matrices)   
        
        #invertibility check
        if not _is_invertibles:
            dataset_info[split]["nonInvertibles"].append(str(_instances.numpy()[0]))
            
        warped_images, _transformed_have_nans = DatasetTools._transformed_images(input_images, 
                                                                                homography_matrices)
        # check for nans in transformed images
        if _transformed_have_nans:
            dataset_info[split]["transformed_have_NaNs"].append(str(_instances.numpy()[0]))
            
#save dataset info
common_utils.save_json(dataset_info, 
                        f"resources/datasets_logs/",
                        f"{dataset}_dataset_info.json"
                        )


[INFO] saving json file to resources/datasets_logs/SkyData_dataset_info.json
[INFO] file saved at resources/datasets_logs.


In [6]:
str(list(dataloader.take(1))[0][0].shape)

'(1, 192, 192, 3)'